<a href="https://colab.research.google.com/github/hslau44/Turing-RSS-Health-Data-Lab-Biomedical-Acoustic-Markers-Notebook-Version/blob/main/Demonstration_COVID_Confounders_edited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Audio-based AI classifiers show no evidence of improved COVID-19 screening over simple symptoms checkers - Demonstration Script

**warning: this is NOT a COVID-19 diagnostic test, this demo is purely educational, does not provide any medical recommendation nor should any action be taken following use,  it serves as a supporting document for the paper.**

This is demonstration of the main inference step, we also provide some additional explanable AI findings not included in the main paper.


To speed up inference time, now is a good time to select a GPU, achieved through Runtime>Change runtime type and selecting a GPU.

Here is the setup of the code and materials needed to run the program

In [ ]:
from IPython.core.display import os
#@markdown **&larr;&larr;&larr;  Run this cell to setup the code**
# To make plots appear in the notebook!
%matplotlib inline

# packages for recording audio in a notebook
!pip install ffmpeg-python --quiet
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
from scipy.io.wavfile import write as wav_write
import io
import os
from pathlib import Path
import ffmpeg
import matplotlib.pyplot as plt


# packages for the model inference step:
!pip install timm==0.4.5 --quiet

if not os.path.exists('Turing-RSS-Health-Data-Lab-Biomedical-Acoustic-Markers'):
  !git clone --recurse-submodules https://github.com/alan-turing-institute/Turing-RSS-Health-Data-Lab-Biomedical-Acoustic-Markers.git --quiet

import sys
sys.path.append('/content/Turing-RSS-Health-Data-Lab-Biomedical-Acoustic-Markers')
from ssast_ciab.src.finetune.ciab.demonstration import main_demo

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

print("Setup Completed")

### Now it is time to record your audio!

Depending on your selection please follow the below instructions for recording your respiratory sounds:

- **Sentence**: Record a sentence “I love nothing more than an afternoon cream tea.” This sentence contains some key sounds (‘aaah’, ‘oooh’, ‘eee’) which may help to indicate your respiratory health Press the Record button, and read the following sentence: “I love nothing more than an afternoon cream tea.” Use the Stop button to stop recording.

- **Exhalation**: Record a 'ha' sound Please make this
recording in a quiet environment. Press the
Record button, and breathe out loudly three
times, making a ‘ha’ sound, as if you were
trying to fog up a window, or see your breath
in cold weather. Use the Stop button to stop
recording. You will see an audio player, which
you can use to playback your recording. Breathe out loudly three times, making
a ‘ha’ sound, as if you were trying to fog up a
window, or see your breath in cold weather.
Press the Record button to begin recording

- **Cough**: Record a cough. Coughing is a potential risk
to others around you. Make sure you are
alone in a room or vehicle when coughing.
For this recording, move an arm’s length
away from your desktop computer, laptop,
phone or tablet. Press the Record button, and
cough, forcing a cough if it doesn’t come
naturally. Use the Stop button to stop
recording. You will see an audio player, which
you can use to playback your recording. Cough once — with your desktop
computer, laptop, phone or tablet an arm’s
length away from you.

- **Three cough**: Record three coughs Coughing is a potential
risk to others around you. Make sure you are
alone in a room or vehicle when coughing.
For this recording, move an arm’s length
away from your desktop computer, laptop,
phone or tablet. Press the Record button, and
cough three times. Cough three times — with
your desktop computer, laptop, phone or
tablet an arm’s length away from you.

In [ ]:
#@markdown **&larr;&larr;&larr; Select the modality you would like to test from the drop down menu below and run this cell**
modality = 'sentence' #@param ["sentence", "three_cough", "cough", "exhalation"]

In [ ]:
#@markdown **&larr;&larr;&larr; Run this cell will immediately start the recording, press the button to stop the recording, you can rerun this cell to make recording again**
audio, sr = get_audio()
Path("/content/demo").mkdir(parents=True, exist_ok=True)
wav_write('/content/demo/sample_1', sr, audio)

In [ ]:
#@markdown **&larr;&larr;&larr; You can run this cell to see your voice in waveform**

plt.figure(figsize=(15,5))
plt.plot(audio)
plt.show()

In [ ]:
#@markdown **&larr;&larr;&larr; Please run this cell to setup the AI model associated with your selected modality**
import gdown
print(f'Downloading the {modality} modality model')
if modality == 'sentence':
  !gdown --folder https://drive.google.com/drive/folders/1Kb04tyDUaSCUDjLdC4EjzKzSIhMocQnw?usp=share_link --quiet
elif modality == 'exhalation':
  !gdown --folder https://drive.google.com/drive/folders/17GEBwClHYFhKWnZ6DpfhasFfG5RCC6mD?usp=share_link
elif modality == 'cough':
  !gdown --folder https://drive.google.com/drive/folders/1WWw4BNUobhi9Jm9Vl5WhFxLkktIObCtP?usp=share_link
elif modality == 'three_cough':
  !gdown --folder https://drive.google.com/drive/folders/1FszBlApEzEcdVNMKdHZx8i9hYB2pbtH6?usp=share_link
else:
  raise AssertionError(f'{modality} is not a modality which we investigated.')
print('Download Completed')

In [ ]:
#@markdown **&larr;&larr;&larr; Finally run this cell to let the AI to analyse your voice**
main_demo(
    f'/content/{modality}/',
    '/content/demo/sample_1',
    name='sample_1')